In [1]:
import plotly.express as px
from glob import glob
import os
import pandas as pd
import csv
from collections import defaultdict

In [2]:
# import plotly.io as pio
# pio.renderers.default = 'iframe' # or 'notebook' or 'colab' or 'jupyterlab'

In [3]:
DUMP_DIR = '../dumps'
dump = {'dfiv': 'dfiv',
        'dfiv_parcs': 'dfiv_parcs',}

In [4]:
alldict = defaultdict(list)
for key, dir in dump.items():
    m = 0
    for subdir in glob(os.path.join(DUMP_DIR, dir, 'data_size*')):
        params = [param.split(':') for param in subdir.split('/')[-1].split('-')]
        
        if os.path.exists(os.path.join(subdir, 'result.csv')):
            with open (os.path.join(subdir, 'result.csv'), 'r') as f:
                reader = csv.reader(f)
                mse = [float(row[0]) for row in list(reader)]
                n = len(mse)
                m += n

            params = {param[0]: [param[1]] * n for param in params}
            subdict = {**params, 'mse': mse, 'model': [key] * n}
            alldict = {key: subdict[key] + alldict[key] for key in subdict.keys()}

In [5]:
df = pd.DataFrame(alldict)
df

,data_size,rho,mse,model
0,5000,0.9,223.873367,dfiv_parcs
1,5000,0.9,230.456345,dfiv_parcs
2,5000,0.9,200.590942,dfiv_parcs
3,5000,0.9,161.784332,dfiv_parcs
4,5000,0.9,233.797455,dfiv_parcs
...,...,...,...,...
235,1000,0.1,945.748230,dfiv
236,1000,0.1,1251.022095,dfiv
237,1000,0.1,861.942566,dfiv
238,1000,0.1,869.289124,dfiv


In [8]:
fig = px.box(df, x='data_size', y='mse', color='model', log_y=True,
                facet_col='rho', facet_col_wrap=3)
fig.show()